In [1]:
import python_utils as utils
import sys
import pickle
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from scipy.optimize import minimize, curve_fit
from scipy import stats
from statsmodels.stats.multitest import multipletests
from scipy.signal import correlate
from scipy.interpolate import interp1d

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

plt.rcParams['axes.titlesize'] = 16
plt.rcParams['axes.labelsize'] = 15
plt.rcParams['legend.fontsize'] = 13


# SIGMOIDPULSE0_002 t_mid = 13000 duration = 10 s

In [2]:
parseg = 250
overlap = 0.5

neglect_t = 3000
t_mid = 13000
t_mid2 = 2300
sigm_par = 0.002

t_tot = 33000

dd_low = 0.850

function = "sigmoidpulse"

In [3]:
sigm_par_poiss = "{:.5f}".format(sigm_par)

Dd = "0.950"
subnets = "STN"
simulations = [str(i) for i in range(32)]


In [4]:
Dd_values = []
current_value = dd_low
while current_value <= float(Dd):
    Dd_values.append(round(current_value, 4))
    current_value += 0.0025

print(Dd_values)

[0.85, 0.8525, 0.855, 0.8575, 0.86, 0.8625, 0.865, 0.8675, 0.87, 0.8725, 0.875, 0.8775, 0.88, 0.8825, 0.885, 0.8875, 0.89, 0.8925, 0.895, 0.8975, 0.9, 0.9025, 0.905, 0.9075, 0.91, 0.9125, 0.915, 0.9175, 0.92, 0.9225, 0.925, 0.9275, 0.93, 0.9325, 0.935, 0.9375, 0.94, 0.9425, 0.945, 0.9475, 0.95]


In [5]:
simulations_folder_path = f"./build/output/n1/{function}_{t_mid}.00_{sigm_par_poiss}_0.00_{Dd}_1.00_1.00_1.6/"
print(simulations_folder_path)

./build/output/n1/sigmoidpulse_13000.00_0.00200_0.00_0.950_1.00_1.00_1.6/


# EXpected PSD

In [6]:
####################### DATI SIMULAZIONE #######################################
cleaned_path = simulations_folder_path.replace("./build/output/n1/", "").replace("_1.00_1.00_1.6/", "")
filename = f"power_{cleaned_path}_{parseg}.txt"

power = np.loadtxt(simulations_folder_path + filename, skiprows=1)
time = np.arange(0, len(power[0]), 1)

In [7]:
df_attesi = []

for i in range(32):
    # Crea un DataFrame con i dati
    df_i = pd.DataFrame({'Tempo': time, 'Valore': power[i]})

    # Calcola la media mobile e assegna i risultati al tempo centrale
    window_size = parseg
    df_i['Media_Mobile'] = df_i['Valore'].rolling(window=window_size, center=True).mean()

    # Rimuovi le righe con valori mancanti (prime e ultime finestre)
    df_i = df_i.dropna()

    # Rimuovi i primi neglect_t elementi
    df_i = df_i.iloc[neglect_t:]

    # Aggiungi il DataFrame alla lista
    df_attesi.append(df_i)

# Stampa il DataFrame risultante
#print(df_attesi)

In [8]:
media = []
error = []
tt = []

for i in range(0, len(df_attesi[0]['Tempo']), int(parseg - parseg*overlap)): # qui metti 0. overlap
    media.append(np.mean([df_i['Media_Mobile'].iloc[i] for df_i in df_attesi]))
    error.append(np.std([df_i['Media_Mobile'].iloc[i] for df_i in df_attesi]))
    tt.append(i + int(parseg*overlap))


In [9]:
%matplotlib widget

tt = np.array(tt)/1000
plt.figure(figsize=(12, 6))
plt.errorbar(tt, media, yerr=error, fmt='o')
plt.xlabel('Time [s]')
plt.ylabel('mean β PSD [u.a.]')
plt.title('Pulse expected mean β PSD')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Dynamic simulation PSD

In [10]:
####################### DATI SIMULAZIONE #######################################
cleaned_path = simulations_folder_path.replace("./build/output/n1/", "").replace("_1.00_1.00_1.6/", "")
filename = f"all_pow_t{cleaned_path}_{parseg}.txt"

all_pow_t = np.loadtxt(simulations_folder_path + filename, skiprows=1)


In [11]:
pow_t_mean = np.mean(all_pow_t, axis=0)
pow_error = np.std(all_pow_t, axis=0)

num_elements = len(pow_t_mean)
t = np.linspace(parseg*overlap/1000, num_elements * parseg*overlap/1000, num_elements)


## time-pow_t

In [12]:
%matplotlib widget
# Creare un grafico dei valori nel tempo
fig, ax1 = plt.subplots(figsize=(12, 5))

# Utilizza la funzione errorbar per tracciare i dati con errori associati
ax1.errorbar(t, pow_t_mean, yerr=pow_error, fmt='none', capsize=1, color='black', ecolor='black', elinewidth=1)

# Plot dei dati originali
ax1.plot(t, pow_t_mean, color='orange')

# Etichette e legenda per l'asse x e y
ax1.set_xlabel('Time [s]')
ax1.set_ylabel('Mean β PSD [u.a.]')

plt.title(f'Mean β PSD for τ = {1/sigm_par/1000} [s]')# nparseg = {parseg}')
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
%matplotlib widget
# Creare un grafico dei valori nel tempo
fig, ax1 = plt.subplots(figsize=(12, 5))

# Utilizza la funzione errorbar per tracciare i dati con errori associati
ax1.errorbar(t, pow_t_mean, yerr=pow_error, fmt='none', capsize=1, color='black', ecolor='black', elinewidth=1)

# Plot dei dati originali
ax1.plot(t, pow_t_mean, label='Dynamic simulation β PSD', color='orange')
#ax1.set_yscale('log')

# Aggiungi il terzo set di dati con errori associati al grafico esistente
plt.errorbar(tt, media, yerr=error, fmt='o', label='Expected PSD')

# Etichette e legenda per l'asse x e y
ax1.set_xlabel('Time [s]')
ax1.set_ylabel('Mean β PSD [u.a.]')
ax1.legend()  # Assicurati di avere tutte le etichette delle leggende necessarie

plt.title(f'Intensità nel Range β for τ = {1/sigm_par/1000} [s] $y_L$ = {10*(dd_low - 0.85):.2f}')# nparseg = {parseg}')
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# statistica

In [14]:
pow_attesa = []
for i in range(0, len(df_attesi[0]['Tempo']), int(parseg - parseg*overlap)): # qui metti 0. overlap
    pow_attesa.append([df_i['Media_Mobile'].iloc[i] for df_i in df_attesi])

In [15]:
all_pow_t_transposed = list(zip(*all_pow_t))

In [16]:
# Esegui il test t indipendente (two-sample t-test)
t_statistic = []
p_value = []
for i in range(len(all_pow_t_transposed)):
    ts, v = stats.ttest_ind(all_pow_t_transposed[i], pow_attesa[i])
    t_statistic.append(ts)
    p_value.append(v)


In [18]:
p_=multipletests(p_value, alpha=0.05, method='fdr_bh', maxiter=1, is_sorted=False, returnsorted=False)

In [19]:
# %matplotlib widget

# Crea un array di valori temporali
time_values = np.arange(parseg*overlap/1000, len(p_value) * parseg*overlap/1000 + parseg*overlap/1000, parseg*overlap/1000)

# Creare le figure per i p-values
fig, (axPvalue1, axPvalue2) = plt.subplots(nrows=2, ncols=1, figsize=(12, 10))

# Plot per la prima figura
axPvalue1.plot(time_values, pow_t_mean, marker='o', color='orange', markerfacecolor='black', label='Dynamic simulation β PSD')
axPvalue1.errorbar(t, pow_t_mean, yerr=pow_error, fmt='none', capsize=1, color='black', ecolor='black', elinewidth=1)
axPvalue1.errorbar(tt, media, yerr=error, fmt='o', label='expected Mean β PSD')

true_indices = np.where(p_[0])[0]
if true_indices.size > 0:  # Controlla se true_indices contiene almeno un elemento
    axPvalue1.axvline(x=time_values[true_indices[0]], color='green', alpha=0.2, label='p-value < 0.05')
    for index in true_indices:
        axPvalue1.axvline(x=time_values[index], color='green', alpha=0.2)

axPvalue1.text(0.01, 0.95, '(a)', transform=axPvalue1.transAxes, fontsize=16, fontweight='bold', va='top')

#axPvalue1.set_yscale('log')
axPvalue1.set_ylabel('Mean β PSD [u.a.]')
axPvalue1.set_title(f'Simulation and expected values vs P-value  τ = {1/sigm_par/1000} [s] $y_L$ = {10*(dd_low - 0.85):.2f}')# nparseg = {parseg}')
axPvalue1.legend()

# Plot per la seconda figura
axPvalue2.plot(time_values, p_[1], marker='.', linestyle='-', markerfacecolor='black', label='P-values')
axPvalue2.set_yscale('log')
axPvalue2.axhline(y=0.05, color='red', linestyle='-', label='0.05 threshold')

if true_indices.size > 0:  # Ripete il controllo per la seconda figura
    axPvalue2.axvline(x=time_values[true_indices[0]], color='green', alpha=0.2, label='p-value < 0.05')
    for index in true_indices:
        axPvalue2.axvline(x=time_values[index], color='green', alpha=0.2)

axPvalue2.text(0.01, 0.95, '(b)', transform=axPvalue2.transAxes, fontsize=16, fontweight='bold', va='top')

axPvalue2.set_xlabel('Time [s]')
axPvalue2.set_ylabel('P-value')
axPvalue2.legend()

# Mostra le figure
plt.tight_layout()
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …